In [ ]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from scripts.all_imports import *
from scripts.utils import  CONFIG, FILE_PATHS, PATH_ROOT, INPUT_PATH, OUTPUT_PATH

print("Configuration:", CONFIG)
print("Path:", FILE_PATHS)
print("Source:", PATH_ROOT)
print("Input:",INPUT_PATH)
print("Output:",OUTPUT_PATH)
print("File",CONFIG["files"]["file_data_pubmed"])


In [ ]:
logging.info("Application started.")
print(f" paths: Input -> {INPUT_PATH} \n output -> {OUTPUT_PATH}")
df_pubmed = pd.DataFrame()
    
logging.info(f"Output file paths: {FILE_PATHS}")
logging.info(f"Output config: {CONFIG}")

In [ ]:
# Define the file path for the input data. If the key "file_data_input" exists in the configuration,
# its value will be used. Otherwise, the default file name "InputSearch.csv" is used.
# Usage of the function
FILE_INPUT_SEARCH = os.path.join(INPUT_PATH, CONFIG["files"].get("file_data_input", "InputSearchDoi.csv"))
print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_search = load_csv_to_dataframe(FILE_INPUT_SEARCH, ';')

In [ ]:
FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_pubmed", "PubMed_Metadata.csv"))
FILE_INPUT_SEARCH = os.path.normpath(FILE_INPUT_SEARCH)

print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_pubmed = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')

pd.set_option('display.max_columns', None)
df_pubmed.head(2)


In [ ]:
FILE_PARSING_PUBMED = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_data_pubmed", "PubMed_Metadata.txt"))
FILE_PARSING_PUBMED = os.path.normpath(FILE_PARSING_PUBMED)

try:
    if os.path.isfile(FILE_PARSING_PUBMED):
        os.remove(FILE_PARSING_PUBMED)
        print(f"File {FILE_PARSING_PUBMED} has been successfully deleted.")
    else:
        print(f"File {FILE_PARSING_PUBMED} does not exist or is not a valid file.")

    map_to_pubmed_format(df_pubmed, FILE_PARSING_PUBMED)

    print(f"Saving Pubmed file: {FILE_PARSING_PUBMED}")
    logging.info(f"Saving Pubmed file: {FILE_PARSING_PUBMED}") 
    
except Exception as e:
    print(f"An error occurred while trying to delete the file: {e}")

In [ ]:
FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_bibliometrix", "Bibliometrix_Metadata.csv"))
FILE_INPUT_SEARCH = os.path.normpath(FILE_INPUT_SEARCH)

print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_bibliometrix = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')

if df_bibliometrix is None:
    print("Erro: 'df_bibliometrix' está definido como None.")
else:
    print(f"'df_bibliometrix' contém {len(df_bibliometrix)} registros.")

pd.set_option('display.max_columns', None)
df_bibliometrix.head(2)

In [13]:
import pandas as pd

# Copiar o DataFrame de bibliometrix
df_search = df_bibliometrix.copy()

# Remover valores NaN na coluna 'DOI' e 'ArticleTitle'
df_search_clean = df_search.dropna(subset=['DOI', 'ArticleTitle'])

# Verificar duplicatas na coluna 'DOI'
duplicated_dois = df_search_clean[df_search_clean['DOI'].duplicated(keep=False)]

# DataFrame com DOIs duplicados
df_duplicated_dois = duplicated_dois[['DOI']].drop_duplicates()
df_duplicated_dois['count'] = (
    duplicated_dois['DOI']
    .value_counts()
    .reindex(df_duplicated_dois['DOI'])
    .values
)

# Verificar duplicatas na coluna 'ArticleTitle'
duplicated_titles = df_search_clean[df_search_clean['ArticleTitle'].duplicated(keep=False)]

# DataFrame com títulos duplicados
df_duplicated_titles = duplicated_titles[['ArticleTitle']].drop_duplicates()
df_duplicated_titles['count'] = (
    duplicated_titles['ArticleTitle']
    .value_counts()
    .reindex(df_duplicated_titles['ArticleTitle'])
    .values
)

# Verificar duplicatas na combinação 'ArticleTitle' + 'DOI'
duplicated_combinations = df_search_clean[df_search_clean.duplicated(subset=['ArticleTitle', 'DOI'], keep=False)]

# DataFrame com combinações duplicadas
df_duplicated_combinations = (
    duplicated_combinations[['ArticleTitle', 'DOI']]
    .value_counts()
    .reset_index()
    .rename(columns={0: 'count'})
)

# Exibir os resultados no console
print("DOIs Duplicados:")
print(df_duplicated_dois)

print("\nTítulos Duplicados:")
print(df_duplicated_titles)

print("\nCombinações Duplicadas (Título + DOI):")
print(df_duplicated_combinations)

# (Opcional) Salvar os resultados em arquivos CSV
df_duplicated_dois.to_csv("duplicated_dois_pot_process.csv", index=False)
df_duplicated_titles.to_csv("duplicated_titles_pot_process.csv", index=False)
df_duplicated_combinations.to_csv("duplicated_combinations_pot_process.csv", index=False)


DOIs Duplicados:
Empty DataFrame
Columns: [DOI, count]
Index: []

Títulos Duplicados:
Empty DataFrame
Columns: [ArticleTitle, count]
Index: []

Combinações Duplicadas (Título + DOI):
Empty DataFrame
Columns: [ArticleTitle, DOI, count]
Index: []


In [14]:
# Parsing metadata to format df_bibliometrix
FILE_PARSING_BIBLIOMETRIX = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_bibliometrix", "Bibliometrix.csv"))
FILE_PARSING_BIBLIOMETRIX = os.path.normpath(FILE_PARSING_BIBLIOMETRIX)

In [15]:
# Parsing metadata to format df_bibliometrix
FILE_PARSING_BIBLIOMETRIX = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_bibliometrix", "Bibliometrix.csv"))
FILE_PARSING_BIBLIOMETRIX = os.path.normpath(FILE_PARSING_BIBLIOMETRIX)

sample_columns = ['AU', 'AF', 'CR', 'AB', 'C1', 'DE', 'DI', 'DT', 'FU', 'FX', 
                  'ID', 'IS', 'J9', 'JI', 'LA', 'OA', 'PU', 'PY', 'RP', 'SO', 
                  'TC', 'TI', 'UT', 'VL', 'C1raw', 'DB', 'AU_UN', 'AU1_UN', 
                  'AU_UN_NR', 'SR_FULL', 'SR']

#map_pubmed_to_bibliometrix(df_bibliometri,FILE_PARSING_BIBLIOMETRIX)
map_pubmed_to_bibliometrix_extended(df_bibliometrix, FILE_PARSING_BIBLIOMETRIX, sample_columns)

#df_bibliometrix.to_excel(FILE_PARSING_BIBLIOMETRIX, index=False)

logging.info(f"Saving Bibliometrix file: {FILE_PARSING_BIBLIOMETRIX}")    


Arquivo processado salvo em: c:\Fontes\PubMedMetaTools\data\processed\files_type\Bibliometrix_format.xlsx


c:\Fontes\PubMedMetaTools\scripts\mapping.py:559: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(clean_surrogates)


In [16]:
data = df_bibliometrix.copy()
data.fillna("", inplace=True)
data

,PMID,ArticleTitle,JournalTitle,ISOAbbreviation,Country,Volume,Pages,ISSN,Language,AbstractText,CopyrightInformation,DOI,PII,PublicationYear,PublicationSeason,Authors,Affiliations,Keywords,MeshTerms,ChemicalSubstances,GrantIDs,GrantOrganizations,DocumentTypes,Status,LastRevisionDate,CompletionDate,PublicationHistory,ConflictOfInterest,PMCID,Citations
0,39639040,Bidirectional relationship between olfaction a...,NPJ Parkinson's disease,NPJ Parkinsons Dis,United States,10,232,2373-8057,eng,Hyposmia (decreased smell function) is a commo...,© 2024. The Author(s).,10.1038/s41531-024-00838-4,232,2024,,"Kim, Jonggeol Jeffrey JJ; Bandres-Ciga, Sara S...","Preventive Neurology Unit, Wolfson Institute o...",,,,ZIA AG000949; ZO1 AG000535,Intramural NIH HHS; U.S. Department of Health ...,Journal Article,PubMed-not-MEDLINE,2024-12-10,0000-00-00,received: 2023-12-6; accepted: 2024-11-11; med...,Competing interests: Members of the 23andMe Re...,PMC11621548,"Van Den Eeden, S. K. et al. Incidence of Parki..."
1,39632912,Slovak database of speech affected by neurodeg...,Scientific data,Sci Data,England,11,1320,2052-4463,eng,A new Slovak speech database EWA-DB was create...,© 2024. The Author(s).,10.1038/s41597-024-04171-6,1320,2024,,"Rusko, Milan M; Sabo, Róbert R; Trnka, Marián ...",Institute of Informatics of the Slovak Academy...,,Humans; Parkinson Disease;physiopathology; Dat...,,,,Dataset; Journal Article,MEDLINE,2024-12-27,2024-12-05,received: 2024-6-25; accepted: 2024-11-26; med...,Competing interests: The authors declare no co...,PMC11618578,"Dubois, B. et al. Clinical diagnosis of Alzhei..."
2,39629536,Validation of Parkinson's Disease Ascertainmen...,Movement disorders : official journal of the M...,Mov Disord,United States,,,1531-8257,eng,BACKGROUND: Electronic medical record (EMR)-ba...,© 2024 The Author(s). Movement Disorders publi...,10.1002/mds.30075,,2024,,"Goldman, Samuel M SM; Weaver, Frances M FM; Ca...","Division of Occupational, Environmental, and C...",Parkinson's disease; ascertainment; diagnostic...,,,I01 CX002040-01; ; ; ;,U.S. Department of Veterans Affairs; NIH HHS; ...,Journal Article,Publisher,2024-12-04,0000-00-00,revised: 2024-11-5; received: 2024-8-14; accep...,,,"Casey JA, Schwartz BS, Stewart WF, Adler NE. U..."
3,39627178,Cortical complexity alterations in motor subty...,The European journal of neuroscience,Eur J Neurosci,France,60,7249-7262,1460-9568,eng,"Based on motor symptoms, Parkinson's disease (...",© 2024 Federation of European Neuroscience Soc...,10.1111/ejn.16612,,2024,,"Dehghan, Yousef Y; Sarbaz, Yashar Y","Biological Systems Modeling Laboratory, Depart...",Parkinson's disease; cortical folding; postura...,Humans; Parkinson Disease;physiopathology;diag...,,; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ; ...,Michael J. Fox Foundation for Parkinson's Rese...,Journal Article,MEDLINE,2024-12-16,2024-12-16,revised: 2024-10-28; received: 2024-9-7; accep...,,,"Ashburner, J., & Friston, K. J. (2000). Voxel‐..."
4,39625512,Identification of isoAsp7-Aβ as a major Aβ var...,Acta neuropathologica,Acta Neuropathol,Germany,148,78,1432-0533,eng,The formation of amyloid-β (Aβ) aggregates in ...,© 2024. The Author(s).,10.1007/s00401-024-02824-9,78,2024,,"Schrempel, Sarah S; Kottwitz, Anna Katharina A...",Paul Flechsig Institute - Centre of Neuropatho...,Abeta; Alzheimer’s disease; Amyloid-β; Automat...,Humans; Alzheimer Disease;pathology;genetics;m...,Amyloid beta-Peptides; Isoaspartic Acid,"SCHI 1437/4-1; WA 1477/6-6; contracts 4001, 00...",German Research Foundation; German Research Fo...,Journal Article,MEDLINE,2024-12-07,2024-12-03,received: 2024-8-27; accepted: 2024-11-2; revi...,Declarations. Conflict of interest: The author...,PMC11615120,"Abramov E, Dolev I, Fogel H, Ciccotosto GD, Ru..."
5,39613827,MJF-14 proximity ligation assay detects early ...,NPJ Parkinson's disease,NPJ Parkinsons Dis,United States,10,227,2373-8057,eng,α-Synuclein proximity ligation assay (PLA) has...,© 2024. The Author(s).,10.1038/s41531

In [17]:
import pandas as pd
import re

 # Lista de campos extraídos para análise bibliométrica
    # Cada campo representa uma informação específica da base de dados utilizada (e.g., Web of Science, Scopus):
    # AU: Nome(s) do(s) autor(es) do artigo.
    # AF: Nome completo do(s) autor(es), conforme aparece na publicação.
    # CR: Referências citadas no artigo (citações listadas no estilo usado na publicação).
    # AB: Resumo do artigo, descrevendo os principais objetivos e descobertas.
    # C1: Endereços institucionais afiliados ao(s) autor(es), com nomes de instituições e departamentos.
    # DE: Palavras-chave do autor, indicando os principais tópicos abordados no artigo.
    # DI: DOI (Digital Object Identifier), identificador digital único para o artigo.
    # DT: Tipo de documento (e.g., artigo, revisão, conferência).
    # FU: Fontes de financiamento ou agências de fomento que apoiaram o trabalho.
    # FX: Informações adicionais fornecidas no campo de agradecimentos.
    # ID: Palavras-chave atribuídas por indexadores (diferentes das palavras-chave do autor).
    # IS: Número da edição do periódico onde o artigo foi publicado.
    # J9: Nome abreviado do periódico (abreviação padrão).
    # JI: ISSN do periódico (International Standard Serial Number).
    # LA: Idioma em que o artigo foi publicado.
    # OA: Indica se o artigo é de acesso aberto (open access).
    # PU: Editora responsável pela publicação do periódico ou conferência.
    # PY: Ano de publicação do artigo.
    # RP: Endereço de correspondência do autor principal.
    # SO: Nome completo do periódico onde o artigo foi publicado.
    # TC: Número total de citações recebidas pelo artigo.
    # TI: Título do artigo.
    # UT: Número de registro único atribuído pela base de dados (e.g., Web of Science).
    # VL: Volume do periódico onde o artigo foi publicado.
    # C1raw: Campo bruto contendo informações não processadas sobre endereços institucionais (antes da normalização).
    # DB: Base de dados de onde os dados bibliométricos foram extraídos (e.g., Web of Science, Scopus).
    # AU_UN: Instituições afiliadas ao(s) autor(es), após normalização.
    # AU1_UN: Instituição afiliada ao primeiro autor, após normalização.
    # AU_UN_NR: Número de instituições únicas afiliadas ao(s) autor(es).
    # SR_FULL: Referência completa (Source Reference), incluindo título, autor(es), periódico e outros detalhes da publicação.
    # SR: Referência abreviada (citação simplificada usada na análise).

order_column = ['AU', 'AF', 'CR', 'AB', 'C1', 'DE', 'DI', 'DT', 'FU', 'FX', 
                  'ID', 'IS', 'J9', 'JI', 'LA', 'OA', 'PU', 'PY', 'RP', 'SO', 
                  'TC', 'TI', 'UT', 'VL', 'C1raw', 'DB', 'AU_UN', 'AU1_UN', 
                  'AU_UN_NR', 'SR_FULL', 'SR']


mapping = {
    "Authors": "AU",
    "Citations": "CR",
    "AbstractText": "AB",
    "Affiliations": "C1",
    "Keywords": "DE",
    "DOI": "DI",
    "DocumentTypes": "DT",
    "GrantOrganizations": "FU",
    "ChemicalSubstances": "ID",
    "ISOAbbreviation": "J9",
    "ISSN": "JI",
    "Language": "LA",
    "PublicationYear": "PY",
    "JournalTitle": "SO",
    "ArticleTitle": "TI",
    "PMID": "UT",
    "Volume": "VL",
    "PUBMED": "DB",
    "Country": "SO_CO",
    "MeshTerms": "MESH",
    "ISSN": "SN",
    "Pages": "PG",
    "GrantIDs": "GRANT_ID",
}

# Functions for cleaning and processing data
def clean_affiliation(affiliation):
    if not affiliation:
        return ""
    affiliation = re.sub(r"\[.*?\]", "", affiliation)
    return re.sub(r"\s+", " ", affiliation.strip())

# Função para formatar nomes no formato esperado
def format_authors(authors):
    if pd.notna(authors):
        authors_list = authors.split(";")  # Separar autores
        formatted_authors = []
        for author in authors_list:
            parts = author.strip().split(",")  # Separar sobrenome do resto
            if len(parts) == 2:
                surname = parts[0].strip().upper()  # Sobrenome em maiúsculas
                first_initial = parts[1].strip()[0].upper()  # Apenas a primeira inicial
                formatted_authors.append(f"{surname} {first_initial}")
        return ";".join(formatted_authors)
    return ""

# Função para formatar os autores no formato especificado
def format_full_authors(authors):
    if pd.notna(authors):
        authors_list = authors.split(";")  # Separar autores por ponto e vírgula
        formatted_authors = []
        for author in authors_list:
            parts = author.strip().split(",")  # Dividir sobrenome e restante do nome
            if len(parts) == 2:
                surname = parts[0].strip().upper()  # Sobrenome em maiúsculas
                names = parts[1].strip().split()  # Nome completo e possíveis iniciais
                # Combinar nomes intermediários e adicionar ponto às iniciais
                first_name = " ".join(
                    [name.upper() + "." if len(name) == 1 else name.upper() for name in names[:-1]]
                )  # Nomes intermediários com ponto nas iniciais
                last_initial = f"{names[-1].upper()}." if len(names[-1]) == 1 else names[-1].upper()  # Última inicial com ponto
                cleaned_author = f"{surname}, {first_name} {last_initial}".strip()
                formatted_authors.append(cleaned_author)

        # Retornar todos os autores formatados, incluindo os últimos
        return ";".join(formatted_authors)
    return ""


# Dados de exemplo com valores a serem padronizados
data = {
    "DT": [
        "ARTICLE",
        "REVIEW",
        "ARTICLE; EARLY ACCESS",
        "PROCEEDINGS PAPER",
        "ARTICLE; PROCEEDINGS PAPER",
        "MEETING ABSTRACT",
        "LETTER",
        "EDITORIAL MATERIAL",
        "CORRECTION",
        "LETTER; EARLY ACCESS",
        "CONFERENCE PAPER",
        "ERRATUM",
        "NOTE",
        "BOOK CHAPTER"
    ]
}


# Função para realizar o mapeamento
def map_values(value):
    # Caso o valor exista no dicionário, retorna o mapeado; caso contrário, mantém o original
    return mapping.get(value, value)

def generate_short_reference(row):
    first_author = row["Authors"].split(";")[0].strip() if "Authors" in row and pd.notna(row["Authors"]) else "NA"
    #year = str(int(row["PublicationYear"])) if "PublicationYear" in row and pd.notna(row["PublicationYear"]) else "NA"
    year = (
        str(int(row["PublicationYear"])) 
        if "PublicationYear" in row and pd.notna(row["PublicationYear"]) and str(row["PublicationYear"]).strip().isdigit()
        else "NA"
    )
    #year = str(int(row["PublicationYear"])) if "PublicationYear" in row and pd.notna(row["PublicationYear"]) else "NA"
    source = row["JournalTitle"] if "JournalTitle" in row and pd.notna(row["JournalTitle"]) else "NA"
    return f"{first_author}, {year}, {source}"

def clean_surrogates(text):
    if isinstance(text, str):
        return text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    return text

# Normalize data
data = df_bibliometrix.copy()
df_mapped = data.rename(columns=mapping)
#df_mapped

df_mapped.fillna("", inplace=True)

# Remove invalid characters
df_mapped = df_mapped.applymap(clean_surrogates)


data["SR_FULL"] = data.apply(generate_short_reference, axis=1)
data["SR"] = data["SR_FULL"]

if "AU" in df_mapped.columns:
    df_mapped["AF"] = df_mapped["AU"]
    df_mapped["AF"] = df_mapped["AF"].apply(format_full_authors)

    df_mapped["AU"] = df_mapped["AU"].apply(format_authors)

if "FU" in df_mapped.columns:
    df_mapped["GRANT_ORG"] = df_mapped["FU"]

if "VL" in df_mapped.columns:
    df_mapped["IS"] = df_mapped["VL"]

if "DT" in df_mapped.columns:
    df_mapped["DT"] = df_mapped["DT"].apply(map_values)

df_mapped["FX"] = ""
df_mapped["OA"] = ""
df_mapped["PU"] = ""
df_mapped["RP"] = ""
df_mapped["TC"] = ""


# Verifica quais colunas de 'order_column' estão presentes em 'df_mapped'
ordered_columns = [col for col in order_column if col in df_mapped.columns]

# Reorganiza as colunas no DataFrame
df_ordered = df_mapped[ordered_columns]
df_ordered

#df_ordered.to_excel(FILE_PARSING_BIBLIOMETRIX, index=False, engine='openpyxl')
#print(f"Arquivo processado salvo em: {FILE_PARSING_BIBLIOMETRIX}")


C:\Users\ceagre\AppData\Local\Temp\ipykernel_14172\1406128667.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_mapped = df_mapped.applymap(clean_surrogates)


,AU,AF,CR,AB,C1,DE,DI,DT,FU,FX,ID,IS,J9,LA,OA,PU,PY,RP,SO,TC,TI,UT,VL
0,KIM J;BANDRES-CIGA S;HEILBRON K;BLAUWENDRAAT C...,"KIM, JONGGEOL JEFFREY JJ;BANDRES-CIGA, SARA S....","Van Den Eeden, S. K. et al. Incidence of Parki...",Hyposmia (decreased smell function) is a commo...,"Preventive Neurology Unit, Wolfson Institute o...",,10.1038/s41531-024-00838-4,Journal Article,Intramural NIH HHS; U.S. Department of Health ...,,,10,NPJ Parkinsons Dis,eng,,,2024,,NPJ Parkinson's disease,,Bidirectional relationship between olfaction a...,39639040,10
1,RUSKO M;SABO R;TRNKA M;ZIMMERMANN A;MALASCHITZ...,"RUSKO, MILAN M.;SABO, RÓBERT R.;TRNKA, MARIÁN ...","Dubois, B. et al. Clinical diagnosis of Alzhei...",A new Slovak speech database EWA-DB was create...,Institute of Informatics of the Slovak Academy...,,10.1038/s41597-024-04171-6,Dataset; Journal Article,,,,11,Sci Data,eng,,,2024,,Scientific data,,Slovak database of speech affected by neurodeg...,39632912,11
2,GOLDMAN S;WEAVER F;CAO L;GONZALEZ B;STROUPE K;...,"GOLDMAN, SAMUEL M. SM;WEAVER, FRANCES M. FM;CA...","Casey JA, Schwartz BS, Stewart WF, Adler NE. U...",BACKGROUND: Electronic medical record (EMR)-ba...,"Division of Occupational, Environmental, and C...",Parkinson's disease; ascertainment; diagnostic...,10.1002/mds.30075,Journal Article,U.S. Department of Veterans Affairs; NIH HHS; ...,,,,Mov Disord,eng,,,2024,,Movement disorders : official journal of the M...,,Validation of Parkinson's Disease Ascertainmen...,39629536,
3,DEHGHAN Y;SARBAZ Y,"DEHGHAN, YOUSEF Y.;SARBAZ, YASHAR Y.","Ashburner, J., & Friston, K. J. (2000). Voxel‐...","Based on motor symptoms, Parkinson's disease (...","Biological Systems Modeling Laboratory, Depart...",Parkinson's disease; cortical folding; postura...,10.1111/ejn.16612,Journal Article,Michael J. Fox Foundation for Parkinson's Rese...,,,60,Eur J Neurosci,eng,,,2024,,The European journal of neuroscience,,Cortical complexity alterations in motor subty...,39627178,60
4,SCHREMPEL S;KOTTWITZ A;PIECHOTTA A;GNOTH K;BÜS...,"SCHREMPEL, SARAH S.;KOTTWITZ, ANNA KATHARINA A...","Abramov E, Dolev I, Fogel H, Ciccotosto GD, Ru...",The formation of amyloid-β (Aβ) aggregates in ...,Paul Flechsig Institute - Centre of Neuropatho...,Abeta; Alzheimer’s disease; Amyloid-β; Automat...,10.1007/s00401-024-02824-9,Journal Article,German Research Foundation; German Research Fo...,,Amyloid beta-Peptides; Isoaspartic Acid,148,Acta Neuropathol,eng,,,2024,,Acta neuropathologica,,Identification of isoAsp7-Aβ as a major Aβ var...,39625512,148
5,JENSEN N;FU Y;BETZER C;LI H;ELFARRASH S;SHAIB ...,"JENSEN, NANNA MØLLER NM;FU, YUHONG Y.;BETZER, ...","Spillantini, M. G. & Goedert, M. The α-synucle...",α-Synuclein proximity ligation assay (PLA) has...,DANDRITE - Danish Research Institute of Transl...,,10.1038/s41531-024-00841-9,Journal Article,Deutsche Forschungsgemeinschaft (German Resear...,,,10,NPJ Parkinsons Dis,eng,,,2024,,NPJ Parkinson's disease,,MJF-14 proximity ligation assay detects early ...,39613827,10
6,WANG Z;WU L;GERASIMENKO M;GILLILAND T;SHAH Z;L...,"WANG, ZERUI Z.;WU, LING L.;GERASIMENKO, MARIA ...","Coughlin D, Irwin DJ. Emerging Diagnostic and ...",BACKGROUND: Tauopathies are a group of age-rel...,"Department of Pathology, Case Western Reserve ...",Alzheimer’s disease; Real-time quaking-induced...,10.1186/s13024-024-00781-1,Journal Article,NIA NIH HHS; Creutzfeldt-Jakob Disease Foundat...,,tau Proteins; Biomarkers,19,Mol Neurodegener,eng,,,2024,,Molecular neurodegeneration,,Seeding activity of skin misfolded tau as a bi...,39609917,19
7,YOON H;KANG D;KIM S,"YOON, HYEMIN H.;KANG, DO-YOUNG DY;KIM, SANGJIN S.","Risacher, S. L. & Saykin, A. J. in Seminars in...",The application of deep learning techniques fo...,"Department of Management Information Systems, ...",Deep learning; Dopamine transporter; Functiona...,10.1038/s41598-024-80938-6,Journal Article,Dong-A University,,,14,Sci Rep,eng,,,2024,,Scientific reports,,Enhancement and evaluation for deep learning-b...,39609597,14
8